In [42]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import sys, os
from data import (
    get_storage_scenario_options,
    get_storage_results,
    calculate_cost_storage_scenarios,
)
from graphs import plot_cost_storage_scenarios, plot_storage_results, plot_renewable_shares



In [12]:
fn_results = "../data/results_storage.parquet"
country = "DE"
start = "2023/01/01 00:00"

In [14]:
df_in = pd.read_parquet(fn_data)
df_in

<class 'pandas.core.frame.DataFrame'>
Index: 1769520 entries, 0 to 1769519
Data columns (total 17 columns):
 #   Column                Dtype         
---  ------                -----         
 0   base                  float64       
 1   wind                  float64       
 2   solar                 float64       
 3   netStorage            float64       
 4   demand                float64       
 5   storageLevel          float64       
 6   MAX_STO               float64       
 7   curtailmentBase       float64       
 8   curtailmentWind       float64       
 9   curtailmentSolar      float64       
 10  share_generation      float64       
 11  share_renewable       float64       
 12  share_storage         float64       
 13  costCurtailNuclear    int64         
 14  costCurtailRenewable  int64         
 15  scenario              object        
 16  date                  datetime64[ns]
dtypes: datetime64[ns](1), float64(13), int64(2), object(1)
memory usage: 243.0+ MB


In [114]:

def plot_renewable_shares(
        df: pd.DataFrame, 
        renewables: list[str]=["wind", "solar"],
        scenario: str="optimalRE",
    ):
    """Plot share of each renewable technology in total renewable generation.
    Args:
        df: dataframe with hourly results
        renewables: list of renewable technologies
        scenario: scenario to plot (usually: optimalRE)
    """
    df_ = (
        df
        [["scenario","share_renewable"] + renewables]
        .groupby(["scenario", "share_renewable"])[renewables]
        .sum()
        .assign(
            total=lambda x: x.sum(1)
        )
    )
    df_ = (df_.div(df_["total"], axis=0) * 100).fillna(0).round(2).reset_index()
    #df_.query("scenario == 'optimalRE'")
    df_p = df_.query("scenario == 'optimalRE'")
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_p["share_renewable"], 
            y=df_p["wind"], 
            mode="lines", 
            name="Wind",
            marker=dict(color="darkgreen"),
        )
    )

    for r in renewables:
        fig.add_trace(
            go.Scatter(
                x=df_p["share_renewable"], 
                y=df_p[r], 
                mode="lines", 
                name=r.capitalize(),
                # marker=dict(color="orange"),
            )
        )
    fig.update_layout(
        title=f"Share of Renewable Technologies in Total Renewable Generation",
        xaxis=dict(domain=[0, 1], title="Share of Renewables in Demand [%]"),
        yaxis=dict(title="Technology Share in Total RE Generation [%]", 
                rangemode="tozero"),
    )
    return fig
fig = plot_renewable_shares(df)
fig.show()

In [112]:
df_ = (
    df
    [["scenario","share_renewable"] + renewables]
    .groupby(["scenario", "share_renewable"])[renewables]
    .sum()
    .assign(
        total=lambda x: x.sum(1)
    )
)
df_ = (df_.div(df_["total"], axis=0) * 100).fillna(0).round(2).reset_index()


{'wind': 72.04, 'solar': 27.96}